## 2. Giải quyết vấn đề

In [1]:
import pandas as pd
import numpy as np

### 2.1. Dữ liệu nhà hàng

#### 2.1.1. File `air_visit_data.csv`

- **Input:** File air_visit_data thu thập những dữ liệu về khách hàng đến cửa hàng theo ngày (nếu ngày mà cửa hàng không có khách thì file sẽ không hiển thị)
- **Output:** Dữ liệu khách hàng của cửa hàng sẽ được tính tổng theo từng ngày
- **Phương pháp:** chúng ta sẽ gom nhóm theo ngày và tính tổng số lượng khách của cửa hàng theo ngày đó, nếu ngày nào không có khách thì sẽ gắn số khách là 0
- **Mục đích:** tạo một chuỗi thời gian liên tục cho mỗi cửa hàng, từ đó m hình có thể học được xu hướng và mô hình vắng khách hoặc đóng cửa của nhà hàng 

In [2]:
path1 = 'data/air_visit_data.csv'
air_visit = pd.read_csv(path1)
air_visit.head(10)

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6
5,air_ba937bf13d40fb24,2016-01-19,9
6,air_ba937bf13d40fb24,2016-01-20,31
7,air_ba937bf13d40fb24,2016-01-21,21
8,air_ba937bf13d40fb24,2016-01-22,18
9,air_ba937bf13d40fb24,2016-01-23,26


In [3]:
air_visit.index = pd.to_datetime(air_visit['visit_date'])
air_visit = air_visit.groupby('air_store_id').apply(lambda data: data['visitors'].resample('1d').sum(), include_groups=False).reset_index()
air_visit['visit_date'] = air_visit['visit_date'].dt.strftime('%Y-%m-%d') #chuyen dang YYYY-mm-dd
air_visit.replace(0, np.nan, inplace=True)
air_visit['was_nil'] = air_visit['visitors'].isnull()
air_visit.fillna({'visitors': 0}, inplace = True) #ignore

air_visit.head(10)

,air_store_id,visit_date,visitors,was_nil
0,air_00a91d42b08b08d9,2016-07-01,35.0,False
1,air_00a91d42b08b08d9,2016-07-02,9.0,False
2,air_00a91d42b08b08d9,2016-07-03,0.0,True
3,air_00a91d42b08b08d9,2016-07-04,20.0,False
4,air_00a91d42b08b08d9,2016-07-05,25.0,False
5,air_00a91d42b08b08d9,2016-07-06,29.0,False
6,air_00a91d42b08b08d9,2016-07-07,34.0,False
7,air_00a91d42b08b08d9,2016-07-08,42.0,False
8,air_00a91d42b08b08d9,2016-07-09,11.0,False
9,air_00a91d42b08b08d9,2016-07-10,0.0,True


##

#### 2.1.2 `File date_info.csv`

- **Input:** file cung cấp cho chúng ta về thứ ngày tháng năm, với nhãn holiday_flg nhắm đánh dấu ngày đó có phải là ngày lễ không
- **Output:** sẽ có thêm 2 cột là 'prev_holiday_day' và 'after_holiday_day'
- **Phương pháp:** chúng ta sẽ thêm 2 biến 'prev_holiday_day' và 'after_holiday_day' với giá trị 0 để đánh dấu là ngày thường và 1 là ngày lễ 
- **Mục đích:** để mô hình có thể học được rằng đó là ngày thường hay lễ và liệu có phải là lễ dài hay không

In [4]:
date_info = pd.read_csv('data/date_info.csv')
date_info.rename(columns = {'calendar_date' : 'visit_date', 'holiday_flg' : 'is_holiday'}, inplace = True)
date_info['prev_holiday'] = date_info['is_holiday'].shift().fillna(0)
date_info['after_holiday'] = date_info['is_holiday'].shift(-1).fillna(1)
date_info.head()

,visit_date,day_of_week,is_holiday,prev_holiday,after_holiday
0,2016-01-01,Friday,1,0.0,1.0
1,2016-01-02,Saturday,1,1.0,1.0
2,2016-01-03,Sunday,1,1.0,0.0
3,2016-01-04,Monday,0,1.0,0.0
4,2016-01-05,Tuesday,0,0.0,0.0


#### 2.1.3 `File air_store_info.csv`

- **Output:** sẽ cho chúng ta dữ liệu về cửa hàng với những dữ liệu như vị trí ...
- **Mục đích:** mô hình sẽ học về những thứ liên quan đến cửa hàng như vị trí ... để có thể thông qua thông tin của hàng để có thể đoán tốt hơn

In [5]:
path3 = 'weather/air_store_info_with_nearest_active_station.csv'
air_store_info = pd.read_csv(path3)
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,"""35.6580681""","""139.7515992""",tokyo__tokyo-kana__tonokyo,35.691667,139.750000,3.730672,3.739835


#### 2.1.4 `File submission.csv`

- **Input:** cho ta dữ liệu một tập test
- **Output:**  cho ta dữ liệu với cột 'air_store_id' sẽ là id của cửa hàng và 'visit_date' là ngày có khách
- **Phương pháp:** dùng str.slice để cắt id cửa hàng và ngày có khách từ cột id

In [6]:
submission = pd.read_csv('data/sample_submission.csv')
submission['air_store_id'] = submission['id'].str.slice(0,20)
submission['visit_date'] = submission['id'].str.slice(21)
submission['visit_date'] = pd.to_datetime(submission['visit_date']).dt.strftime('%Y-%m-%d')
submission['visitors'] = np.nan #để bỏ qua cột này
submission['is_test'] = True
submission['is_number'] = submission.index

submission.head()


,id,visitors,air_store_id,visit_date,is_test,is_number
0,air_00a91d42b08b08d9_2017-04-23,NaN,air_00a91d42b08b08d9,2017-04-23,True,0
1,air_00a91d42b08b08d9_2017-04-24,NaN,air_00a91d42b08b08d9,2017-04-24,True,1
2,air_00a91d42b08b08d9_2017-04-25,NaN,air_00a91d42b08b08d9,2017-04-25,True,2
3,air_00a91d42b08b08d9_2017-04-26,NaN,air_00a91d42b08b08d9,2017-04-26,True,3
4,air_00a91d42b08b08d9_2017-04-27,NaN,air_00a91d42b08b08d9,2017-04-27,True,4


In [7]:
#xóa cột id
submission = submission.drop(['id'], axis = 'columns')
print(submission.head())

   visitors          air_store_id  visit_date  is_test  is_number
0       NaN  air_00a91d42b08b08d9  2017-04-23     True          0
1       NaN  air_00a91d42b08b08d9  2017-04-24     True          1
2       NaN  air_00a91d42b08b08d9  2017-04-25     True          2
3       NaN  air_00a91d42b08b08d9  2017-04-26     True          3
4       NaN  air_00a91d42b08b08d9  2017-04-27     True          4


##

#### 2.1.5 Merge dataset

- **Input:** ta sẽ có các file train như `air_visit.csv`, `air_store_info.csv` và file test `submission.csv`
- **Output:** ta sẽ nhận được một file gộp lại
- **Mục đích:** để khi đưa vào mô hình thì mô hình sẽ thông qua các thuộc tính để học và dự đoán

In [21]:
data = pd.concat((air_visit, submission))
print(data['is_test'].dtype)
data['is_test']  = data['is_test'].fillna(False).astype('bool') #vì cột khi ép có type là object, dùng astype để chuyển sang boolean
data = pd.merge(left = data, right = date_info, on = 'visit_date', how = 'left')
data = pd.merge(left = data, right = air_store_info, on = 'air_store_id', how = 'left')
data['visitors'] = data['visitors'].astype(float)
data.info()

object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328298 entries, 0 to 328297
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   air_store_id          328298 non-null  object 
 1   visit_date            328298 non-null  object 
 2   visitors              296279 non-null  float64
 3   was_nil               296279 non-null  object 
 4   is_test               328298 non-null  bool   
 5   is_number             32019 non-null   float64
 6   day_of_week           328298 non-null  object 
 7   is_holiday            328298 non-null  int64  
 8   prev_holiday          328298 non-null  float64
 9   after_holiday         328298 non-null  float64
 10  air_genre_name        328298 non-null  object 
 11  air_area_name         328298 non-null  object 
 12  latitude              328298 non-null  float64
 13  longitude             328298 non-null  float64
 14  latitude_str          328298 non-null  object